In [ ]:
#Codigo para postar o dado do mqtt para o banco de dados Mysql
import mysql.connector             #pip install mysql-connector-python
import paho.mqtt.client as mqtt    #pip install mysql-connector-python paho-mqtt
import json

# Configurações do MySQL
DB_CONFIG = {
    "host": "localhost",  # Alterar para o endereço do servidor MySQL
    "port":3306,
    "user": "root",  # Substituir pelo usuário do banco
    "password": "admin",  # Substituir pela senha do banco
    "database": "teste"  # Substituir pelo nome do banco
}

# Configurações do MQTT
MQTT_BROKER = "broker.emqx.io"  # Substituir pelo endereço do broker MQTT
MQTT_PORT = 1883  # Porta padrão do MQTT
MQTT_TOPIC = "ldr/readings"  # Tópico para receber as leituras


def insert_ldr_reading(reading):
    try:
        connection = mysql.connector.connect(**DB_CONFIG)
        cursor = connection.cursor()

        query = "INSERT INTO ldr_readings (red, green, blue) VALUES (%s, %s, %s)"
        cursor.execute(query, tuple(reading))
        connection.commit()
        print(f"Leitura {reading} inserida com sucesso!")
    except mysql.connector.Error as err:
        print(f"Erro ao inserir leitura no banco de dados: {err}")
    finally:
        cursor.close()
        connection.close()

def on_message(client, userdata, msg):
    try:
        # Decodificando a mensagem recebida
        payload = msg.payload.decode("utf-8")
        reading = json.loads(payload)  # JSON no formato {"red": 0.5, "green": 0.2, "blue": 0.8}

        red = reading.get("red")
        green = reading.get("green")
        blue = reading.get("blue")
        print(blue,red,green)
        if red is not None and green is not None and blue is not None:
           
            insert_ldr_reading([red, green, blue])
        else:
            print(f"Dados inválidos recebidos: {payload}")
    except json.JSONDecodeError as e:
        print(f"Erro ao decodificar JSON: {e}")
    except Exception as e:
        print(f"Erro ao processar mensagem: {e}")


def main():
    client = mqtt.Client()
    client.on_message = on_message

    try:
        
        client.connect(MQTT_BROKER, MQTT_PORT, 60)

        client.subscribe(MQTT_TOPIC)
        print(f"Assinado no tópico {MQTT_TOPIC} e aguardando mensagens...")

        # Iniciar o loop para receber mensagens
        client.loop_forever()
    except Exception as e:
        print(f"Erro na conexão com o MQTT: {e}")

if __name__ == "__main__":
    main()
